In [1]:
import os

In [2]:
%pwd

'e:\\Working Project\\New folder\\Kidney-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Working Project\\New folder\\Kidney-Disease-Classification'

In [6]:
!pip install dagshub

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached lxml-5.3.0-cp311-cp311-win_amd64.whl.metadata (3.9 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
   ---------------------------------------- 0.0/258.2 kB ? eta -:--:--
   ------ --------------------------------- 41.0/258.2 kB 1.9 MB/s eta 0:00:01
   ---------------------- ----------------- 143.4/258.2 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------  256.0/258.2 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------  256.0/258.2 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 258.2/258.2 kB 1.2 MB/s eta 0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
   ---------------------------------------- 0.0/139.2 kB ? eta -:--:--
   -------------------------------------- - 133.1/139.2 kB 7.7 MB/s eta 0:00:01
   ---------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import dagshub
dagshub.init(repo_owner='AshishSMehra', repo_name='my-first-repo', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "AshishSMehra/my-first-repo"

Repository AshishSMehra/my-first-repo initialized!

🏃 View run intelligent-bat-787 at: https://dagshub.com/AshishSMehra/my-first-repo.mlflow/#/experiments/0/runs/451ffb2f54a942ce8e299c4066e156ee
🧪 View experiment at: https://dagshub.com/AshishSMehra/my-first-repo.mlflow/#/experiments/0


In [10]:
import tensorflow as tf

In [11]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [13]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [14]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/entbappy/Kidney-Disease-Classification-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [15]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [16]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-01-18 12:59:09,806: INFO: common: yaml file: config\config.yaml loaded successfully:]


[2025-01-18 12:59:09,917: INFO: common: yaml file: params.yaml loaded successfully:]
[2025-01-18 12:59:09,921: INFO: common: created directory at: artifacts:]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 73s 7s/step - loss: 6.2579 - accuracy: 0.5180
[2025-01-18 13:00:26,445: INFO: common: json file saved at: scores.json:]


2025/01/18 13:00:27 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-01-18 13:00:31,721: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.:]
INFO:tensorflow:Assets written to: C:\Users\HI\AppData\Local\Temp\tmpagu1qx69\model\data\model\assets
[2025-01-18 13:00:35,177: INFO: builder_impl: Assets written to: C:\Users\HI\AppData\Local\Temp\tmpagu1qx69\model\data\model\assets:]


2025/01/18 13:02:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run smiling-newt-285 at: https://dagshub.com/AshishSMehra/my-first-repo.mlflow/#/experiments/0/runs/60036dbe450343b598da9f31b44db4d7
🧪 View experiment at: https://dagshub.com/AshishSMehra/my-first-repo.mlflow/#/experiments/0


MlflowException: API request to endpoint /api/2.0/mlflow/registered-models/create failed with error code 403 != 200. Response body: ''